In [17]:
#PREPROCESS 1
from alpaca.data.requests import StockBarsRequest
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.timeframe import TimeFrameUnit
from datetime import datetime

import pandas as pd
import numpy as np
global min_date
global max_date
import pandas as pd
import alpaca.data.historical as alp
client = alp.StockHistoricalDataClient(api_key="AKH3ZFJ2LWAKH7SSV2AHKD4PK4", secret_key="7gGJDh5UAxh3gg8V21d3AcuPfNJcwtKqukiqMhdhsjrW")
min_date=datetime.now().date()+pd.Timedelta(days=-720)
max_date=datetime.now().date()+pd.Timedelta(days=-1)
def getMinuteLevel(asset):
    global min_date
    global max_date
    request_params = StockBarsRequest(
      symbol_or_symbols=[asset],
      timeframe=TimeFrame(1, TimeFrameUnit.Minute),
      #timeframe=TimeFrame(59, TimeFrameUnit.Minute),
      start=min_date,#dt(2024,1,1,0,0,0),
      end=max_date#dt(2024,11,1,0,0,0)
    )
    btc_bars = client.get_stock_bars(request_params)
    #print('done with call')
    # Convert to dataframe
    bars_df=btc_bars.df
    #print(bars_df)
    bars_df=bars_df.reset_index()
    bars_df['date']=[str(stamp)[:10] for stamp in bars_df.timestamp]
    bars_df['hour']=[int(str(stamp)[11:13]) for stamp in bars_df.timestamp]
    bars_df['minute']=[int(str(stamp)[14:16]) for stamp in bars_df.timestamp]
    bars_df['hour_mins']=[60*hr for hr in bars_df.hour]
    
    bars_df['day_min']=[hr_mins+m for hr_mins,m in zip(list(bars_df.hour_mins),list(bars_df.minute))]
    #bars_df['sum_of_price']=[vol*vwap for vol,vwap in zip(list(bars_df.volume),list(bars_df.vwap))]
    #bars_df['price_set']=[[vwap]*int(vol) for vol,vwap in zip(list(bars_df.volume),list(bars_df.vwap))]
    bars_df=bars_df.drop(columns=['hour','minute','hour_mins'])
    #bars_df['timestamp']=pd.to_datetime(bars_df['timestamp'], errors='coerce')
    return bars_df

#getMinuteLevel('BHST')

In [19]:
#PREPROCESS 2: 
from holidays import US
import pandas as pd
import numpy as np
import yfinance as yf # Uncommented this so the code runs
from itertools import combinations

global verbose
verbose = False

# --- RSI FUNCTION DEFINITION ---
def calculate_rsi(series, window, min_lag):
    """Calculates the Relative Strength Index (RSI)."""
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Use min_periods=1 to maximize data availability, span=window for smoothing
    avg_gain = gain.ewm(span=window, min_periods=min_lag).mean()
    avg_loss = loss.ewm(span=window, min_periods=min_lag).mean()
    
    # Handle division by zero
    rs = np.divide(avg_gain, avg_loss, out=np.zeros_like(avg_gain), where=avg_loss!=0)
    rsi = 100 - (100 / (1 + rs))
    return rsi

# --- AGGREGATION FUNCTION ---
def getAggPizza(s, level):
    pza = getMinuteLevel(s)
    print('got minute-level')
   
    pza['current_vwap'] = pza['vwap'].ffill()
    pza['vwap_volume'] = pza['current_vwap'] * pza['volume']
   
    # Filter Holidays and Market Hours
    us_holidays = US(years=range(2023, 2026))
    pza['date'] = pza['timestamp'].dt.date
    pza['is_weekday'] = pza['timestamp'].dt.weekday < 5
    pza['is_not_holiday'] = ~pza['date'].isin([d for d in us_holidays])
   
    # Convert to ET for accurate local market hours (handles DST automatically)
    pza['timestamp_et'] = pza['timestamp'].dt.tz_convert('America/New_York')
    pza['time_et'] = pza['timestamp_et'].dt.time  # Separate column for time comparison
   
    market_hours = (pza['time_et'] >= pd.to_datetime('09:30').time()) & \
                   (pza['time_et'] < pd.to_datetime('16:00').time())
   
    pza = pza[pza['is_weekday'] & pza['is_not_holiday'] & market_hours].drop(
        columns=['date', 'is_weekday', 'is_not_holiday', 'time_et']
    ).reset_index(drop=True)  # Keep timestamp_et for flooring
   
    print('after weekends')
   
    # Floor the full datetime (in ET) to create the aggregation level
    pza['timestamp_level'] = pza['timestamp_et'].dt.floor(f'{level}min')
   
    pza_agg = pza.groupby('timestamp_level').agg({
        'vwap_volume': 'sum',
        'volume': 'sum'
    }).reset_index()
   
    pza_agg['mean_vwap'] = pza_agg['vwap_volume'] / pza_agg['volume']
    pza_agg['mean_vwap'] = pza_agg['mean_vwap'].ffill()
    pza_agg = pza_agg.rename(columns={'mean_vwap': f'mean_vwap_{level}'}).drop(columns=['vwap_volume'])
   
    # Extract hour/minute/day_of_week from the floored ET timestamp
    pza_agg['hour'] = pza_agg['timestamp_level'].dt.hour
    pza_agg['minute'] = pza_agg['timestamp_level'].dt.minute
    pza_agg['day_of_week'] = pza_agg['timestamp_level'].dt.dayofweek
   
    # minutes_since_open: 9:30 ET = 0
    pza_agg['minutes_since_open'] = (pza_agg['hour'] - 9) * 60 + (pza_agg['minute'] - 30)
    pza_agg['session_progress'] = pza_agg['minutes_since_open'] / 390.0
   
    # Optional: drop the helper if not needed downstream
    # pza_agg = pza_agg.drop(columns=['timestamp_et'])  # if you don't need it later
   
    print(level)
    return pza_agg

# --- MAIN FEATURE FUNCTION ---
def getGalledPrices(s, level, leads, lags):
    pza_agg = getAggPizza(s, level)
    
    # Merge with SPY
    spy_agg = getAggPizza('SPY', level)[['timestamp_level', 'mean_vwap_'+str(level)]].rename(
        columns={'timestamp_level': 'timestamp_level_SPY', 'mean_vwap_'+str(level): 'mean_vwap_'+str(level)+'_SPY'}
    )
    
    pza_agg = pd.merge(pza_agg, spy_agg, left_on='timestamp_level', right_on='timestamp_level_SPY', how='right')
    pza_agg['timestamp_level'] = pza_agg['timestamp_level_SPY']
    
    # Calculate Relative Price (Stock / SPY)
    pza_agg['mean_vwap_'+str(level)] = pza_agg['mean_vwap_'+str(level)] / pza_agg['mean_vwap_'+str(level)+'_SPY']
    pza_agg['mean_vwap_'+str(level)] = pza_agg['mean_vwap_'+str(level)].ffill()
    pza_agg['volume'] = pza_agg['volume'].fillna(0)
    pza_agg['pct_change'] = pza_agg['mean_vwap_'+str(level)].pct_change()
    
    lag_ptiles = range(10, 100, 20)
    lead_ptiles = [15, 85]
    
    print(len(pza_agg))
    print('before loop')
    
    max_lag_rows = 100 * (6.5 * 60) / level
    if max_lag_rows > len(pza_agg):
        print('not enough past prices')
        return None
        
    min_lag = min(lags)
    
    # --- MAIN LAG LOOP ---
    for lag in lags:
        lag_rows = int(lag * (6.5 * 60 / level))
        
        # Percentiles
        for p in lag_ptiles:
            pza_agg[f'{p}th_ptile_lag_{lag}days'] = (
                pza_agg['mean_vwap_15'].shift(1)
                .rolling(window=lag_rows, min_periods=min_lag)
                .quantile(p/100)
            )
            pza_agg['change_lag_'+str(lag)+str(p)+'th_ptile'] = (
                pza_agg['mean_vwap_'+str(level)] - pza_agg[f'{p}th_ptile_lag_{lag}days']
            ) / pza_agg[f'{p}th_ptile_lag_{lag}days']
            
        # Volatility & Momentum
        pza_agg['rolling_avg_vol_'+str(lag)] = pza_agg['volume'].shift(1).rolling(window=lag_rows, min_periods=min_lag).mean()
        pza_agg['relative_volume_'+str(lag)] = pza_agg['volume'] / pza_agg['rolling_avg_vol_'+str(lag)]
        
        pza_agg['volatility_'+str(lag)] = pza_agg['pct_change'].shift(1).rolling(window=lag_rows, min_periods=min_lag).std()
        
        rolling_mean = pza_agg['mean_vwap_'+str(level)].shift(1).rolling(window=lag_rows, min_periods=min_lag).mean()
        rolling_std = pza_agg['mean_vwap_'+str(level)].shift(1).rolling(window=lag_rows, min_periods=min_lag).std()
        pza_agg[f'rolling_mean_{lag}'] = rolling_mean
        pza_agg[f'rolling_std_{lag}'] = rolling_std
        
        pza_agg[f'z_score_{lag}'] = (pza_agg['mean_vwap_'+str(level)] - rolling_mean) / rolling_std.replace(0, np.nan)
        pza_agg['bb_width_'+str(lag)] = (rolling_std * 2) / rolling_mean
        
        # Efficiency Ratio
        net_change = pza_agg['mean_vwap_15'].shift(1).rolling(lag_rows, min_periods=min_lag).apply(lambda x: abs(x.iloc[-1] - x.iloc[0]) if len(x) > 1 else np.nan)
        sum_of_changes = pza_agg['mean_vwap_' + str(level)].diff().abs().shift(1).rolling(window=lag_rows, min_periods=min_lag).sum()
        pza_agg[f'efficiency_ratio_{lag}'] = net_change / sum_of_changes.replace(0, np.nan)
        
        # Skewness
        pza_agg[f'skew_{lag}'] = pza_agg['pct_change'].shift(1).rolling(window=lag_rows, min_periods=min_lag).skew()
        
        # Volume Force
        pza_agg[f'volume_force_{lag}'] = pza_agg['pct_change'] * pza_agg[f'relative_volume_{lag}']
        
        # RSI
        pza_agg['rsi_'+str(lag)] = calculate_rsi(pza_agg['mean_vwap_'+str(level)], window=lag_rows, min_lag=min_lag)
        pza_agg[f'rsi_divergence_{lag}'] = np.sign(pza_agg[f'rsi_{lag}'].pct_change()) * np.sign(pza_agg['pct_change'])
        
        # Adjustments
        session_adjust = 1 - np.abs(2 * pza_agg['session_progress'] - 1) + 0.5
        pza_agg[f'session_adj_rel_volume_{lag}'] = pza_agg[f'relative_volume_{lag}'] * session_adjust
        
        weekday_avg_vol = pza_agg.groupby('day_of_week')[f'volatility_{lag}'].transform('mean')
        pza_agg[f'weekday_adj_vol_{lag}'] = pza_agg[f'volatility_{lag}'] / weekday_avg_vol.replace(0, np.nan)
        pza_agg[f'weekday_adj_vol_{lag}'] = pza_agg[f'weekday_adj_vol_{lag}'].fillna(1)

    # Momentum Diff (MM Diff)
    for lag1, lag2 in combinations(lags, 2):
        s_lag, l_lag = sorted((lag1, lag2))
        col_short = f'50th_ptile_lag_{s_lag}days'
        col_long = f'50th_ptile_lag_{l_lag}days'
        
        pza_agg[f'mm_diff_{s_lag}_{l_lag}'] = (pza_agg[col_short] - pza_agg[col_long]) / pza_agg[col_long]
        
        mom_col = f'mm_diff_{s_lag}_{l_lag}'
        vol_col = f'volatility_{l_lag}'
        pza_agg[f'risk_adj_{mom_col}'] = pza_agg[mom_col] / pza_agg[vol_col].replace(0, np.nan)
        pza_agg[f'risk_adj_{mom_col}'] = pza_agg[f'risk_adj_{mom_col}'].fillna(0)


    # Lead (Target) Calculation
    min_lead = min(leads)
    for lead in leads:
        lead_rows = int(lead * (6.5 * 60 / level))
        for p in lead_ptiles:
            pza_agg[f'{p}th_ptile_lead_{lead}days'] = (
                pza_agg['mean_vwap_15']
                .iloc[::-1].shift(1)
                .rolling(window=lead_rows, min_periods=min_lead)
                .quantile(p/100)
                .iloc[::-1]
                .values
            )
            pza_agg['change_lead_'+str(lead)+'_'+str(p)+'th_ptile'] = (
                pza_agg[f'{p}th_ptile_lead_{lead}days'] - pza_agg['mean_vwap_'+str(level)]
            ) / pza_agg['mean_vwap_'+str(level)]

    # --- EARNINGS DATE CALCULATION (OPTIMIZED) ---
    t = yf.Ticker(s)
    earnings_df = pd.DataFrame(t.earnings_dates).reset_index()
    
    # Check if we have data
    if 'Earnings Date' in earnings_df.columns:
        earnings_df = earnings_df.rename(columns={'Earnings Date': 'earning_date'})
    
    # Ensure Timestamp compatibility
   # pza_agg['timestamp_level'] = pd.to_datetime(pza_agg['timestamp_level'], utc=True)
    
    has_earnings = False
    if not earnings_df.empty and 'earning_date' in earnings_df.columns:
        try:
            # Convert earnings dates to Eastern Time to match timestamp_level
            earnings_df['earning_date'] = pd.to_datetime(earnings_df['earning_date']).dt.tz_convert('America/New_York')
            earnings_df = earnings_df.sort_values('earning_date').dropna(subset=['earning_date'])
            pza_agg = pza_agg.sort_values('timestamp_level')
            has_earnings = True
        except Exception as e:
            print(f"Earnings date conversion failed: {e}")
    if has_earnings:
        # Backward Search (Days Since)
        pza_agg = pd.merge_asof(
            pza_agg,
            earnings_df[['earning_date']],
            left_on='timestamp_level',
            right_on='earning_date',
            direction='backward'
        ).rename(columns={'earning_date': 'last_earnings_date'})

        # Forward Search (Days Before)
        pza_agg = pd.merge_asof(
            pza_agg,
            earnings_df[['earning_date']],
            left_on='timestamp_level',
            right_on='earning_date',
            direction='forward'
        ).rename(columns={'earning_date': 'next_earnings_date'})

        pza_agg['days_since_earnings'] = ((pza_agg['timestamp_level'] - pza_agg['last_earnings_date']).dt.total_seconds() / 86400).fillna(90.0)
        pza_agg['days_before_earnings'] = ((pza_agg['next_earnings_date'] - pza_agg['timestamp_level']).dt.total_seconds() / 86400).fillna(90.0)
        
        pza_agg = pza_agg.drop(columns=['last_earnings_date', 'next_earnings_date'])
    else:
        print('No earnings data found or processing failed.')
        pza_agg['days_since_earnings'] = 90.0
        pza_agg['days_before_earnings'] = 90.0


    # --- INTERACTIVE FEATURES SECTION ---
    
    # 1. Earnings Interaction
    target_lag = lags[0]
    
    if 'days_before_earnings' in pza_agg.columns:
        days_to_earn = pza_agg['days_before_earnings'].clip(lower=0)
        earnings_proximity = 1 / (days_to_earn + 1)
        
        base_metrics = ['volatility', 'relative_volume', 'z_score', 'bb_width', 'efficiency_ratio', 'rsi']
        
        for metric in base_metrics:
            col_name = f'{metric}_{target_lag}'
            if col_name in pza_agg.columns:
                pza_agg[f'{metric}_{target_lag}_x_earn_prox'] = (
                    pza_agg[col_name] * earnings_proximity
                )

        pct_col_name = f'change_lag_{target_lag}50th_ptile' 
        if pct_col_name in pza_agg.columns:
            pza_agg[f'change_lag_{target_lag}_50th_ptile_x_earn_prox'] = (
                pza_agg[pct_col_name] * earnings_proximity
            )

        # 2. Day of Week Interactions
        # Add the pct change column to the list for this loop
        loop_metrics = base_metrics + [pct_col_name] if pct_col_name in pza_agg.columns else base_metrics

        for metric in loop_metrics:
            if metric == pct_col_name:
                col_name = metric
                short_name = 'pct_chg_50th'
            else:
                col_name = f'{metric}_{target_lag}'
                short_name = metric

            if col_name in pza_agg.columns:
                # Mon = 0, Fri = 4
                pza_agg[f'{short_name}_Mon_Interact'] = np.where(pza_agg['day_of_week'] == 0, pza_agg[col_name], 0)
                pza_agg[f'{short_name}_Fri_Interact'] = np.where(pza_agg['day_of_week'] == 4, pza_agg[col_name], 0)

        # 3. Time of Day Interactions
        pza_agg['is_open_30m'] = (pza_agg['minutes_since_open'] <= 30).astype(int)
        pza_agg['is_close_30m'] = (pza_agg['minutes_since_open'] >= 360).astype(int)
        pza_agg['is_mid_day'] = ((pza_agg['minutes_since_open'] > 30) & (pza_agg['minutes_since_open'] < 360)).astype(int)

        for metric in loop_metrics:
            if metric == pct_col_name:
                col_name = metric
                short_name = 'pct_chg_50th'
            else:
                col_name = f'{metric}_{target_lag}'
                short_name = metric

            if col_name in pza_agg.columns:
                pza_agg[f'{short_name}_Open30m'] = pza_agg[col_name] * pza_agg['is_open_30m']
                pza_agg[f'{short_name}_Close30m'] = pza_agg[col_name] * pza_agg['is_close_30m']
                pza_agg[f'{short_name}_MidDay'] = pza_agg[col_name] * pza_agg['is_mid_day']

    # At the end of getGalledPrices, just before return pza_agg
    pza_agg['timestamp_level'] = pza_agg['timestamp_level'].dt.tz_localize(None)  # or .dt.tz_convert(None)
    return pza_agg


s='TSM'
# Assuming yf is available, we run the function to test
ggp=getGalledPrices(s,15,[15],[10,25,50,75]) 
ggp.columns = [col + '_' + str(s) for col in ggp.columns]
ggp

got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop


C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pza_agg[f'weekday_adj_vol_{lag}'] = pza_agg[f'volatility_{lag}'] / weekday_avg_vol.replace(0, np.nan)
C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:170: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pza_agg[f'mm_diff_{s_lag}_{l_lag}'] = (pza_agg[col_short] - pza_agg[col_long]) / pza_agg[col_long]
C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:174: PerformanceWarning: DataFrame is h

,timestamp_level_TSM,volume_TSM,mean_vwap_15_TSM,hour_TSM,minute_TSM,day_of_week_TSM,minutes_since_open_TSM,session_progress_TSM,timestamp_level_SPY_TSM,mean_vwap_15_SPY_TSM,...,bb_width_MidDay_TSM,efficiency_ratio_Open30m_TSM,efficiency_ratio_Close30m_TSM,efficiency_ratio_MidDay_TSM,rsi_Open30m_TSM,rsi_Close30m_TSM,rsi_MidDay_TSM,pct_chg_50th_Open30m_TSM,pct_chg_50th_Close30m_TSM,pct_chg_50th_MidDay_TSM
0,2024-01-29 09:30:00,921284.0,0.240588,9.0,30.0,0.0,0.0,0.000000,2024-01-29 09:30:00-05:00,487.852201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-29 09:45:00,448030.0,0.240097,9.0,45.0,0.0,15.0,0.038462,2024-01-29 09:45:00-05:00,487.907719,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-29 10:00:00,416533.0,0.238710,10.0,0.0,0.0,30.0,0.076923,2024-01-29 10:00:00-05:00,487.972007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-29 10:15:00,304973.0,0.238920,10.0,15.0,0.0,45.0,0.115385,2024-01-29 10:15:00-05:00,487.815952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-29 10:30:00,231276.0,0.238933,10.0,30.0,0.0,60.0,0.153846,2024-01-29 10:30:00-05:00,487.562156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12813,2026-01-16 14:45:00,192413.0,0.495268,14.0,45.0,4.0,315.0,0.807692,2026-01-16 14:45:00-05:00,692.807822,...,0.051369,0.0,0.000000,0.100406,0.0,0.000000,58.242469,0.0,0.000000,0.046626
12814,2026-01-16 15:00:00,385631.0,0.495122,15.0,0.0,4.0,330.0,0.846154,2026-01-16 15:00:00-05:00,692.692390,...,0.051603,0.0,0.000000,0.102041,0.0,0.000000,58.173099,0.0,0.000000,0.046293
12815,2026-01-16 15:15:00,403161.0,0.494722,15.0,15.0,4.0,345.0,0.884615,2026-01-16 15:15:00-05:00,692.494719,...,0.051827,0.0,0.000000,0.098801,0.0,0.000000,57.981603,0.0,0.000000,0.045364
12816,2026-01-16 15:30:00,447989.0,0.495100,15.0,30.0,4.0,360.0,0.923077,2026-01-16 15:30:00-05:00,692.092262,...,0.000000,0.0,0.096644,0.000000,0.0,58.112989,0.000000,0.0,0.046046,0.000000


In [21]:
[c for c in ggp.columns if 'rolling_mean_' in c]

['rolling_mean_10_TSM',
 'rolling_mean_25_TSM',
 'rolling_mean_50_TSM',
 'rolling_mean_75_TSM']

In [29]:
all=pd.read_excel('all_stocks_class.xlsx')
swa_stocks=all[all['inds']=='Semiconductors']

In [31]:
#PREPROCESS 3: get ggp table for each stock
import time
start_time = time.time()
global level
industry = 'semiconductors'
level=15
#industry='restaurants'
for s in swa_stocks.stocks:
    #print(s)
    #print(type(s))
    print(s=='nan')
    start_time_loop = time.time()
    try:
        s=s.replace('-', '.')
    except:
        continue
    if isinstance(s, float):
        continue
    try:
        #pd.read_csv('ggp_rel_tables/'+industry+'/ggp_'+s+'.csv')
        buudfoon
        continue
    except:
        print(s)
        ggp=getGalledPrices(s,15,[15],[10,25,50])
        try:
            ggp.columns = [col + '_' + str(s) for col in ggp.columns]
            ggp.to_csv('ggp_rel_tables/'+industry+'/ggp_'+s+'.csv')
        except:
            continue
    end_time_loop = time.time()
    elapsed_time = end_time_loop - start_time_loop
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

False
WKEY
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 137.49 seconds
False
IMOS
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 125.12 seconds
False
ICG
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 128.13 seconds
False
UMC
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 181.77 seconds
False
MX
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 149.72 seconds
False
NVEC
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 128.89 seconds
False
POWI
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 159.09 seconds
False
SLAB
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed t

MOBXW: $MOBXW: possibly delisted; no earnings dates found
C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:283: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pza_agg[f'{short_name}_Mon_Interact'] = np.where(pza_agg['day_of_week'] == 0, pza_agg[col_name], 0)
C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:284: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pza_agg[f'{short_name}_Fri_Interact'] = np.where(pza_agg['day_of_week'] == 4, pza_agg[col_name], 0)
C:\Users\allth\AppData\Local\Temp\ipykernel_21

No earnings data found or processing failed.
Elapsed time: 109.69 seconds
False
MOBX
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop


MOBX: $MOBX: possibly delisted; no earnings dates found
C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:283: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pza_agg[f'{short_name}_Mon_Interact'] = np.where(pza_agg['day_of_week'] == 0, pza_agg[col_name], 0)
C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:284: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pza_agg[f'{short_name}_Fri_Interact'] = np.where(pza_agg['day_of_week'] == 4, pza_agg[col_name], 0)
C:\Users\allth\AppData\Local\Temp\ipykernel_2175

No earnings data found or processing failed.
Elapsed time: 126.16 seconds
False
SIMO
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 134.95 seconds
False
SITM
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 138.15 seconds
False
SKYT
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 155.71 seconds
False
SMTC
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 178.20 seconds
False
STM
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 209.75 seconds
False
SWKS
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 189.97 seconds
False
SYNA
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 153.85 seconds
False
TSEM
got minute-level
aft

LAES: $LAES: possibly delisted; no earnings dates found
C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:283: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pza_agg[f'{short_name}_Mon_Interact'] = np.where(pza_agg['day_of_week'] == 0, pza_agg[col_name], 0)
C:\Users\allth\AppData\Local\Temp\ipykernel_21752\2949348379.py:284: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pza_agg[f'{short_name}_Fri_Interact'] = np.where(pza_agg['day_of_week'] == 4, pza_agg[col_name], 0)
C:\Users\allth\AppData\Local\Temp\ipykernel_2175

No earnings data found or processing failed.
Elapsed time: 207.77 seconds
False
WOLF
got minute-level
after weekends
15
got minute-level
after weekends
15
12818
before loop
Elapsed time: 202.65 seconds
Elapsed time: 11387.26 seconds


In [33]:
#PREPROCESS 4: get join_df based on ggp tables

import glob
import pandas as pd
import os


folder_path = "ggp_rel_tables/" + industry

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {folder_path}")

print(f"Found {len(csv_files)} files: { [os.path.basename(f) for f in csv_files] }")

# Load the first file as the base (to get the master timestamp index)
base_file = csv_files[0]
base_stock = os.path.basename(base_file).replace('ggp_', '').replace('.csv', '')  # e.g., 'NVDA'
base_df = pd.read_csv(base_file).drop(columns=['Unnamed: 0'], errors='ignore')  # Drop unnamed index if present
print(base_stock)

# Identify the timestamp column (assumes format: timestamp_level_{stock})
timestamp_col_base = f'timestamp_level_{base_stock}'
if timestamp_col_base not in base_df.columns:
    raise KeyError(f"Timestamp column {timestamp_col_base} not found in {base_file}")

# Set the base timestamp as index
join_df = base_df.set_index(timestamp_col_base)

# List to collect all timestamp columns for coalescing later
timestamp_cols = [timestamp_col_base]

# Loop over the remaining files and outer-join them
for file_path in csv_files[1:]:
    stock = os.path.basename(file_path).replace('ggp_', '').replace('.csv', '')
    print(f"Joining {stock}...")

    df = pd.read_csv(file_path).drop(columns=['Unnamed: 0'], errors='ignore')  # Drop unnamed index

    timestamp_col = f'timestamp_level_{stock}'
    if timestamp_col not in df.columns:
        print(f"Warning: Timestamp column {timestamp_col} missing in {file_path}, skipping")
        continue

    # Set this stock's timestamp as index and rename to match the base index name
    df = df.set_index(timestamp_col)
    df.index.name = join_df.index.name  # critical for alignment

    # Outer join on the datetime index
    join_df = join_df.join(df, how='outer')

    # Add to list for coalescing
    timestamp_cols.append(timestamp_col)

# Sort by timestamp for chronological order
join_df = join_df.sort_index()

# Create 'timestamp' as coalesce of all timestamp_cols (now they are columns in join_df)
timestamp_cols_in_df = [col for col in timestamp_cols if col in join_df.columns]
if timestamp_cols_in_df:
    join_df['timestamp'] = join_df[timestamp_cols_in_df].bfill(axis=1).iloc[:, 0]
    # Drop the individual ts cols
    join_df = join_df.drop(columns=timestamp_cols_in_df)
else:
    print("Warning: No timestamp columns found for coalescing")

# Optional: reset index if you want timestamp as a column
# join_df = join_df.reset_index()

print(f"\nFinal joined DataFrame shape: {join_df.shape}")
join_df.to_csv('feature_presentation_'+industry+'.csv')
#print(join_df.head())

Found 64 files: ['ggp_ADI.csv', 'ggp_AIP.csv', 'ggp_ALAB.csv', 'ggp_ALGM.csv', 'ggp_AMD.csv', 'ggp_AOSL.csv', 'ggp_ARM.csv', 'ggp_ASX.csv', 'ggp_AVGO.csv', 'ggp_CEVA.csv', 'ggp_CRDO.csv', 'ggp_CRUS.csv', 'ggp_DIOD.csv', 'ggp_GCTS.csv', 'ggp_GFS.csv', 'ggp_GSIT.csv', 'ggp_HIMX.csv', 'ggp_ICG.csv', 'ggp_IMOS.csv', 'ggp_INTC.csv', 'ggp_LAES.csv', 'ggp_LASR.csv', 'ggp_LEDS.csv', 'ggp_LSCC.csv', 'ggp_MCHP.csv', 'ggp_MOBX.csv', 'ggp_MOBXW.csv', 'ggp_MPWR.csv', 'ggp_MRAM.csv', 'ggp_MRVL.csv', 'ggp_MTSI.csv', 'ggp_MU.csv', 'ggp_MX.csv', 'ggp_MXL.csv', 'ggp_NVDA.csv', 'ggp_NVEC.csv', 'ggp_NVTS.csv', 'ggp_NXPI.csv', 'ggp_ON.csv', 'ggp_PI.csv', 'ggp_POET.csv', 'ggp_POWI.csv', 'ggp_PRSO.csv', 'ggp_PXLW.csv', 'ggp_QCOM.csv', 'ggp_QRVO.csv', 'ggp_QUIK.csv', 'ggp_RMBS.csv', 'ggp_SIMO.csv', 'ggp_SITM.csv', 'ggp_SKYT.csv', 'ggp_SLAB.csv', 'ggp_SMTC.csv', 'ggp_STM.csv', 'ggp_SWKS.csv', 'ggp_SYNA.csv', 'ggp_TSEM.csv', 'ggp_TSM.csv', 'ggp_TXN.csv', 'ggp_UMC.csv', 'ggp_VLN.csv', 'ggp_VSH.csv', 'ggp_WKEY.cs

In [35]:
df2=pd.read_csv('ggp_rel_tables/semiconductors/ggp_TSM.csv')
[c for c in df2.columns if 'rolling_mean_' in c]

['rolling_mean_10_TSM', 'rolling_mean_25_TSM', 'rolling_mean_50_TSM']

In [37]:
#PREPROCESS 5: get jino and identify featrues/targets
def get_features(s):
    lags = [10, 25, 50, 75]  # lag days
    ptiles = [10, 30, 50, 70, 90]  # percentiles
    other_lag_metrics = [
        'rolling_avg_vol', 'relative_volume', 'volatility', 'z_score',
        'bb_width', 'efficiency_ratio', 'skew', 'volume_force',
        'rsi', 'rsi_divergence', 'session_adj_rel_volume', 'weekday_adj_vol'
    ]

    lag_features = []

    # 1. Change-from-percentile features: 5 per lag × 4 lags = 20
    for lag in lags:
        for p in ptiles:
            lag_features.append(f'change_lag_{lag}{p}th_ptile_{s}')

    # 2. Other lag metrics: 12 per lag × 4 lags = 48
    for lag in lags:
        for metric in other_lag_metrics:
            lag_features.append(f'{metric}_{lag}_{s}')

    # Total lag features: 20 + 48 = 68
    # (We removed the 20 raw ptile_lag columns as they are time-correlated)

    # Add timestamp column
    #lag_features.append(f'timestamp_level_{s}')

    print(f"Total features generated: {len(lag_features)}")  # Should be 69

    return lag_features

def get_non_feature_vars(s):
    lags = [10, 25, 50, 75]  # lag days
    ptiles = [10, 30, 50, 70, 90]  # percentiles
    lag_ptile_vars=[]
    lag_vars=[]
    for lag in lags:
        for p in ptiles:
            lag_ptile_vars.append(f'lag_{lag}{p}th_ptile_{s}')
    
        lag_vars.append(f'rolling_mean_{lag}_{s}')
        lag_vars.append(f'rolling_std_{lag}_{s}')
    non_feature_vars=lag_ptile_vars+lag_vars
    return non_feature_vars
# Example usage
features = get_features('NVDA')
#non_features=get_non_feature_vars('NVDA')
#LOAD JINO AND ISOLATE TRAINING TIMES
# 1. Load the full wide table
jino = pd.read_csv('feature_presentation_' + industry + '.csv')
ts_cols=[c for c in jino.columns if 'timestamp' in c]
timestamp_col=ts_cols[-1]
jino = jino.sort_values(timestamp_col).reset_index(drop=True)
# 2. Replicate your splitting logic exactly
rows_per_day = 26  
max_lead_days = 15  
buffer_rows = max_lead_days * rows_per_day

total_rows = len(jino)
test_size = int(total_rows * 0.2)
val_size = int(total_rows * 0.2)

# Calculate the end of the training section
idx_test_start = total_rows - test_size - buffer_rows
idx_val_end = idx_test_start - buffer_rows
idx_val_start = idx_val_end - val_size
idx_train_end = idx_val_start - buffer_rows

# 3. Filter jino to only include the Training Range
# This ensures feature selection (correlations) has zero look-ahead bias
jino_train = jino.iloc[:idx_train_end].copy()

print(f"Original rows: {total_rows}")
print(f"Training rows for feature selection: {len(jino_train)}")

#GET FEATURES AND TARGETS
import pandas as pd
jino_cols=jino.columns

stocks=pd.read_excel('all_stocks_class.xlsx')
ind_stocks=stocks[stocks['inds'].str.lower()==industry].stocks
get_rats=jino[[c for c in jino.columns if 'change_lead' in c]].columns
#get_rats
features_giant=[]
for s in ind_stocks:
    #print(s)
    features_giant+=get_features(s)

features_giant = [col for col in features_giant if 'nan' not in col]

Total features generated: 68
Original rows: 12818
Training rows for feature selection: 6522
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total features generated: 68
Total fea

In [ ]:
#FEATURE SELECTION 1: GET SPEARMAN CORRS
import pandas as pd
from scipy.stats import spearmanr
import time

start_time = time.time()
# 1. Initialize empty list to store results (much faster than appending to a DF)
results_list = []
iter=0
for t in get_rats:
    start_time_loop=time.time()
    #if iter==0:
    print(t)
    for v in features_giant:
        if iter==0:
            print(v)
        # 2. Get spearman corr
        # Assumes 'stocks' is your main DataFrame
        # .dropna() is important as spearmanr can return NaN if data is missing
        if v in jino_cols:
            corr, p_value = spearmanr(jino[t], jino[v], nan_policy='omit')
        
        # 3. Add [t, v, corr] to the list
        results_list.append({'target': t, 'potential_var': v, 'corr': corr})
    end_time_loop = time.time()
    elapsed_time = end_time_loop - start_time_loop
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    iter+=1

# 4. Create the DF from the list
spearman_or_fruit = pd.DataFrame(results_list)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
# 5. Filter for correlations greater than .05 (absolute value is usually better)
# Using .abs() ensures you don't accidentally throw away strong negative correlations
filtered_results = spearman_or_fruit[spearman_or_fruit['corr'].abs() > 0.05]
filtered_results

spearman_or_fruit.to_excel('spearman_or_fruit_'+industry+'.xlsx')

change_lead_15_15th_ptile_ADI
change_lag_1010th_ptile_WKEY
change_lag_1030th_ptile_WKEY
change_lag_1050th_ptile_WKEY
change_lag_1070th_ptile_WKEY
change_lag_1090th_ptile_WKEY
change_lag_2510th_ptile_WKEY
change_lag_2530th_ptile_WKEY
change_lag_2550th_ptile_WKEY
change_lag_2570th_ptile_WKEY
change_lag_2590th_ptile_WKEY
change_lag_5010th_ptile_WKEY
change_lag_5030th_ptile_WKEY
change_lag_5050th_ptile_WKEY
change_lag_5070th_ptile_WKEY
change_lag_5090th_ptile_WKEY
change_lag_7510th_ptile_WKEY
change_lag_7530th_ptile_WKEY
change_lag_7550th_ptile_WKEY
change_lag_7570th_ptile_WKEY
change_lag_7590th_ptile_WKEY
rolling_avg_vol_10_WKEY
relative_volume_10_WKEY
volatility_10_WKEY
z_score_10_WKEY
bb_width_10_WKEY
efficiency_ratio_10_WKEY
skew_10_WKEY
volume_force_10_WKEY
rsi_10_WKEY
rsi_divergence_10_WKEY
session_adj_rel_volume_10_WKEY
weekday_adj_vol_10_WKEY
rolling_avg_vol_25_WKEY
relative_volume_25_WKEY
volatility_25_WKEY
z_score_25_WKEY
bb_width_25_WKEY
efficiency_ratio_25_WKEY
skew_25_WKEY
vol

In [4]:
# #FEATURE SELECTION 2: SELECT PREDS FOR EACH STOCK
import glob
import pandas as pd
industry = 'semiconductors'
corrs_df=pd.read_excel(f'spearman_or_fruit_{industry}.xlsx')
folder_path = "ggp_rel_tables/"+industry

# Get all CSV files in the folder
csv_files = glob.glob(folder_path + "/*.csv")
stocks=[]
for f in csv_files:
    #print(f)
    filename = f.split('\\')[-1].split('/')[-1]

    # 2. Remove prefix and suffix
    ticker = filename.replace('ggp_', '').replace('.csv', '')
    stocks.append(ticker)



def getPredSet(s, ind, tar, cutoff, corrs, top_n=27):
    # 1. Base logic: Right target + High enough correlation
    # 2. Add Filter: Exclude anything containing 'ALAB'
    logic = (
        (corrs['target'] == tar) & 
        (corrs['corr'].abs() > cutoff) & 
        (~corrs['potential_var'].str.contains('ALAB', case=False))
    )
    
    # 3. Sort by strength so we keep the most useful ones
    filtered = corrs[logic].copy()
    filtered['abs_corr'] = filtered['corr'].abs()
    filtered = filtered.sort_values('abs_corr', ascending=False)
    
    # 4. Return as a list, capped at top_n
    pred_set = filtered['potential_var'].head(top_n).tolist()
    
    return pred_set

predictors = getPredSet('WKEY', 'semiconductors', 'change_lead_15_85th_ptile_WKEY', 0.1,corrs_df)
print(f"Found {len(predictors)} features for this target.")
print(predictors)

Found 27 features for this target.
['volatility_25_GFS', 'weekday_adj_vol_25_GFS', 'volatility_50_GFS', 'weekday_adj_vol_50_GFS', 'skew_75_MRAM', 'bb_width_50_IMOS', 'bb_width_50_NVDA', 'rolling_avg_vol_25_GFS', 'weekday_adj_vol_25_INTC', 'volatility_25_INTC', 'weekday_adj_vol_50_ASX', 'volatility_50_ASX', 'bb_width_25_NVDA', 'volatility_10_VSH', 'change_lag_7510th_ptile_ARM', 'efficiency_ratio_75_HIMX', 'weekday_adj_vol_10_VSH', 'skew_10_SITM', 'efficiency_ratio_75_TSEM', 'bb_width_25_IMOS', 'change_lag_5070th_ptile_SITM', 'bb_width_25_CEVA', 'skew_50_SMTC', 'change_lag_7530th_ptile_ARM', 'z_score_50_SITM', 'rolling_avg_vol_25_TXN', 'change_lag_7530th_ptile_QCOM']


In [7]:
#MODEL TRAINING 1: DEFINE TIME METRICS FXN AND LOAD DATA TO SPLIT
import numpy as np
import pandas as pd

def get_temporal_metrics(y_true, y_pred, timestamps):
    """
    Calculates extended metrics including temporal variance of True Positives.
    timestamps: Must be a pandas Series of datetime objects corresponding to y_true/y_pred
    """
    # 1. Basic Counts
    total_rows = len(y_true)
    class_1_count = np.sum(y_true == 1)
    class_1_pct = (class_1_count / total_rows) * 100 if total_rows > 0 else 0
    
    # 2. Identify True Positives (TP)
    # Ensure inputs are aligned numpy arrays or pandas series
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    timestamps = np.array(timestamps)
    
    tp_mask = (y_true == 1) & (y_pred == 1)
    tp_timestamps = timestamps[tp_mask]
    
    # 3. Calculate Temporal Aggregates for TPs
    if len(tp_timestamps) > 1:
        # Convert to numeric (seconds) to calculate variance
        # Note: timestamps must be datetime64[ns]
        ts_numeric = tp_timestamps.astype('datetime64[s]').astype('int64')
        
        # Variance (in Days Squared for readability)
        # 86400 seconds in a day
        var_seconds_sq = ts_numeric.var()
        var_days_sq = var_seconds_sq / (86400 ** 2)
        
        # Range (Duration between first and last TP)
        time_range = pd.to_datetime(tp_timestamps.max()) - pd.to_datetime(tp_timestamps.min())
        range_days = time_range.total_seconds() / 86400
        
        # Mean Time Distance (Average gap between consecutive TPs)
        # Sort first to measure gaps
        sorted_ts = np.sort(ts_numeric)
        avg_gap_hours = np.mean(np.diff(sorted_ts)) / 3600
        
    else:
        var_days_sq = 0
        range_days = 0
        avg_gap_hours = 0

    return {
        "total_test_rows": total_rows,
        "test_class1_pct": class_1_pct,
        "tp_var_days_sq": var_days_sq,
        "tp_range_days": range_days,
        "tp_avg_gap_hours": avg_gap_hours,
        "n_true_positives": len(tp_timestamps)
    }
# Load Data
try:
    #df = def getPredSet(s, ind, tar, cutoff,corrs):
    df=pd.read_csv('feature_presentation_'+industry+'.csv')
except FileNotFoundError:
    try:
        df = pd.read_csv(f)
    except FileNotFoundError:
        print(f"File not found for {s}. Skipping.")


In [8]:
import joblib
import os

# --- NEW: SAVE CHAMPION MODEL ---
# Create directory if it doesn't exist
MODEL_DIR = "./models/"
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [15]:
#MODEL TRAINING 2: THOMAS AND HIS FIENDS
# CLASS GRID GIANT PRUNING
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, precision_score, precision_recall_fscore_support,f1_score
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Set style
plt.style.use('ggplot')

# -----------------------------------
all_results = []
num = 0
IMPORTANCE_THRESHOLD = 0.025 # Define the pruning threshold

# 5. Define Parameter Grid
param_grid_values = {
    'n_estimators': [300],
    'learning_rate': [0.1],
    'max_depth': [7],
    'min_samples_leaf': [20]
}

# Assuming csv_files and industry are defined elsewhere in your environment
# for f in csv_files:
# (I am assuming the loop starts here based on your snippet)

for s in stocks:
    for p in ['15','85']:
        start_time = time.time()
        skip = False
        if s[0] in ['A','B']:
            continue
        num += 1
        print(s)
        if num >= 10:
            print(num)
            break
        # 1. Setup and Load Data
        
        #if s not in ['ARCO','BH','BJRI','BROS','CAVA','CBRL','CMG','CNNE']:
        #continue
        if '.' in s or not s:
            continue
        
        print(f"\n================ Starting analysis for symbol: {s} ================")
        
        
        # Parameterize timestamp column and sort
        timestamp_col = f'timestamp_level_SPY_{s}'
        if timestamp_col not in df.columns:
            print(f"Timestamp column {timestamp_col} not found. Skipping {s}.")
            continue

        if not pd.api.types.is_datetime64_any_dtype(df[timestamp_col]):
            print(f"Converting {timestamp_col} to datetime...")
            df[timestamp_col] = pd.to_datetime(df[timestamp_col], utc=True)
        df = df.sort_values(timestamp_col).reset_index(drop=True)
        
        # 2. Set up Features and Targets
        # Wrapped in parentheses to allow multi-line list addition
        target_col_name = f'change_lead_15_{p}th_ptile_{s}'
        initial_features = getPredSet(s, industry, target_col_name, .05,corrs_df)
        
        feature_cols_all = initial_features + [timestamp_col]
    
        
        if target_col_name not in df.columns:
            print(f"Target column {target_col_name} not found. Skipping {s}.")
            continue
        
        # 3. Create Binary Target and Clean NaNs
        if p == '85':
            df['target_class'] = (df[target_col_name] > 0.03).astype(int) # Upside
        else:
            df['target_class'] = (df[target_col_name] < -0.03).astype(int) # Downside
        print(f"Using threshold: {.03} for {s} with lead_ptile {p}")
        
        # model_df contains all features + timestamp + target, only non-NaN rows
        model_df = df[feature_cols_all + ['target_class']].copy()
        model_df[feature_cols_all] = model_df[feature_cols_all].ffill() # Fill features
        model_df = model_df.dropna()
        if len(model_df) < 1000:
            print(f"Not enough data points ({len(model_df)}) after cleanup. Skipping {s}.")
            continue
        
        # 4. Define and Apply 70/15/15 Split with Buffer (Indices remain fixed)
        #print('before splitting')
        rows_per_day = 26  
        max_lead_days = 15  
        buffer_rows = max_lead_days * rows_per_day
        
        total_rows = len(model_df)
        test_size = int(total_rows * 0.2)
        val_size = int(total_rows * 0.2)
        
        idx_test_start = total_rows - test_size - buffer_rows
        idx_val_end = idx_test_start - buffer_rows
        idx_val_start = idx_val_end - val_size
        idx_train_end = idx_val_start - buffer_rows
        
        # Initial X/y split (still contains ALL possible features + timestamp)
        X_pool = model_df[feature_cols_all]
        y_pool = model_df['target_class']
        
        # ---------------------------------------------------------
        # === RECURSIVE FEATURE PRUNING LOOP ===
        # ---------------------------------------------------------
        #print('after splitting')
        current_features = initial_features.copy()
        temp_clf = None 
        best_params = {}
        
        # --- NEW: Track the "Champion" (Best Parsimonious) Set ---
        champion_features = current_features.copy()
        champion_params = {}
        champion_score = -1.0   # The score of the current champion
        global_max_score = -1.0 # The absolute best score seen (to calculate the 0.01 drop)
        # --------------------------------------------------------
        
        iteration = 0
        print('before while true')
        while True:
            iteration += 1
            
            # 4b. Slice data based on CURRENT features
            current_cols = current_features + [timestamp_col]
            X = X_pool[current_cols]
            y = y_pool
            
            X_train = X.iloc[:idx_train_end]
            y_train = y.iloc[:idx_train_end]
            X_val = X.iloc[idx_val_start:idx_val_end]
            y_val = y.iloc[idx_val_start:idx_val_end]
            print(len(X_val))
            X_train_clean = X_train.drop(columns=[timestamp_col])
            X_val_clean = X_val.drop(columns=[timestamp_col])
            
            if len(X_train_clean) <= 0 or len(X_val_clean) <= 0:
                print("ERROR: Training or Validation set is empty after slicing. Breaking.")
                skip = True
                break
            
            # 6. Grid Search (Re-run for current feature set)
            iter_best_score = -1
            iter_best_params = {}
            
            keys, values = zip(*param_grid_values.items())
            print('before val loop')
            for v in product(*values):
                params = dict(zip(keys, v))
                clf = GradientBoostingClassifier(random_state=42, **params)
                clf.fit(X_train_clean, y_train)
                print('after clf fit')
                preds_val = clf.predict(X_val_clean)
                print('after clf val predict')
                # Calculate score for this specific param combo
                current_score = f1_score(y_val, preds_val, pos_label=1, zero_division=0)#precision_score(y_val, preds_val, zero_division=0)
               
                if current_score > iter_best_score:
                    iter_best_score = current_score
                    iter_best_params = params
            
            print(f"Iteration {iteration}: Features={len(current_features)} | Best Val Score={iter_best_score:.4f}")
            
            # --- NEW: Check if this feature set is the new "Champion" ---
            # Update the Global Max Score if this iteration beat everything before it
            if iter_best_score > global_max_score:
                global_max_score = iter_best_score
            
            # Parsimony Logic: 
            # We accept this new set if the score hasn't dropped by more than 0.01 compared 
            # to the absolute best score seen so far.
            if iter_best_score >= (global_max_score - 0.01):
                champion_features = current_features.copy()
                champion_params = iter_best_params
                champion_score = iter_best_score
                print(f"   >>> New Champion Selected! (Score: {iter_best_score:.4f} is within 0.01 of Max {global_max_score:.4f})")
            else:
                print(f"   >>> Score drop > 0.01 (Val: {iter_best_score:.4f} vs Max: {global_max_score:.4f}). Keeping previous Champion.")
            # ------------------------------------------------------------
            
            # 7. Train Model for Importance Check (using Train+Val)
            X_train_final = pd.concat([X_train_clean, X_val_clean])
            y_train_final = pd.concat([y_train, y_val])
            
            # We use the params found in THIS iteration to determine importances for pruning
            temp_clf = GradientBoostingClassifier(random_state=42, **iter_best_params)
            temp_clf.fit(X_train_final, y_train_final)
            
            # 8. Check and Prune Features
            importances = temp_clf.feature_importances_
            
            next_features = [
                feat for feat, imp in zip(current_features, importances)
                if imp >= IMPORTANCE_THRESHOLD
            ]
            
            # --- Convergence and Safeguard Checks ---
            if set(next_features) == set(current_features):
                print(f"Iteration {iteration}: Converged (No features dropped).")
                break
               
            if len(next_features) == 0:
                 print(f"Iteration {iteration}: All features below threshold. Breaking.")
                 break
                 
            if iteration > 5:
                print(f"Iteration {iteration}: Max iterations reached.")
                break
            
            print(f"Iteration {iteration}: Pruning {len(current_features) - len(next_features)} features.")
            current_features = next_features
            
        
        if skip == False:
        # ---------------------------------------------------------
        # === RESTORE CHAMPION SET ===
        # ---------------------------------------------------------
            print(f"\nPruning Complete. Reverting to Champion Feature Set (Size: {len(champion_features)}) with Score: {champion_score:.4f}")
            current_features = champion_features
            best_params = champion_params
        
        # Retrain Final Model on the Champion Features + Champion Params
        X_train_final_pool = X_pool[current_features + [timestamp_col]].iloc[:idx_val_end]
        y_train_final_pool = y_pool.iloc[:idx_val_end]
        X_train_final_clean = X_train_final_pool.drop(columns=[timestamp_col])
        
        final_clf = GradientBoostingClassifier(random_state=42, **best_params)
        final_clf.fit(X_train_final_clean, y_train_final_pool)
        
        print(f"\nGrid Search & Selection Complete. Best Parameters: {best_params}")
        
        # 8. Final Evaluation on Hold-out Test Set
        # Slice the test set using the FINAL 'current_features' (which is now the Champion set)
        X_test = X_pool[current_features + [timestamp_col]].iloc[idx_test_start:]
        y_test = y_pool.iloc[idx_test_start:]
        
        X_test_clean = X_test.drop(columns=[timestamp_col])
        preds_test = final_clf.predict(X_test_clean)
        
        print(f"\n--- Final Test Set Evaluation for {s} ---")
        print(classification_report(y_test, preds_test, zero_division=0))
        
        # 9. Extract and Store Results
        prec, r, f1, support = precision_recall_fscore_support(y_test, preds_test, average=None, zero_division=0)
        
        class_1_precision = prec[1]
        class_1_recall = r[1]
        class_1_f1 = f1[1]
        
        # NOTE: Ensure get_temporal_metrics is defined in your environment
        try:
            timestamps = X_test[timestamp_col] 
            metrics = get_temporal_metrics(y_test, preds_test, timestamps)
        except NameError:
            metrics = {}
            print("Warning: get_temporal_metrics function not found.")
        
        # 10. Feature Importance (Using the final feature set and model)
        importance_df = pd.DataFrame({
            'Feature': current_features, 
            'Importance': final_clf.feature_importances_
        }).sort_values('Importance', ascending=False)
    
        top_importance_df = importance_df[importance_df['Importance'] > 0.05]
        top_importance_dict = top_importance_df.set_index('Feature')['Importance'].to_dict()
        
        print("\nTop 10 Feature Importances (Final Tuned Model):")
        print(importance_df.head(10).to_markdown(index=False))
        
        result_entry = {
            'Symbol': s,
            'Final_Feature_Count': len(current_features),
            'Best_Params': str(best_params),
            'Test_Precision_Class1': class_1_precision,
            'Test_Recall_Class1': class_1_recall,
            'Test_F1_Class1': class_1_f1,
            'Top_Feature_Importances': top_importance_dict
        }
        result_entry = {**result_entry, **metrics}
        all_results.append(result_entry)

        # Bundle the model and the specific features used
        model_data = {
            'model': final_clf,
            'features': current_features,  # These are the champion features
            'params': best_params,
            'symbol': s,
            'side': p,  # '15' for downside, '85' for upside
            'trained_at': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')
        }
        
        # Save the file
        save_path = f"{MODEL_DIR}champion_model_{s}_{p}.pkl"
        joblib.dump(model_data, save_path, compress=3)
        
        print(f"✅ Model saved to: {save_path}")

# --- LOOP END: CONSOLIDATE RESULTS ---
results_df = pd.DataFrame(all_results)
print("\n================== CONSOLIDATED RESULTS ==================")
print(results_df.to_markdown(index=False))

CEVA

================ Starting analysis for symbol: CEVA ================
Converting timestamp_level_SPY_CEVA to datetime...
Using threshold: 0.03 for CEVA with lead_ptile 15
before while true
2556
before val loop
after clf fit
after clf val predict
Iteration 1: Features=27 | Best Val Score=0.8384
   >>> New Champion Selected! (Score: 0.8384 is within 0.01 of Max 0.8384)
Iteration 1: Pruning 16 features.
2556
before val loop
after clf fit
after clf val predict
Iteration 2: Features=11 | Best Val Score=0.8120
   >>> Score drop > 0.01 (Val: 0.8120 vs Max: 0.8384). Keeping previous Champion.
Iteration 2: Converged (No features dropped).

Pruning Complete. Reverting to Champion Feature Set (Size: 27) with Score: 0.8384

Grid Search & Selection Complete. Best Parameters: {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 20}

--- Final Test Set Evaluation for CEVA ---
              precision    recall  f1-score   support

           0       0.64      0.52      

C:\Users\allth\AppData\Local\Temp\ipykernel_23628\770689073.py:28: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ts_numeric = tp_timestamps.astype('datetime64[s]').astype('int64')


Using threshold: 0.03 for CEVA with lead_ptile 85
before while true
2556
before val loop
after clf fit
after clf val predict
Iteration 1: Features=27 | Best Val Score=0.6408
   >>> New Champion Selected! (Score: 0.6408 is within 0.01 of Max 0.6408)
Iteration 1: Pruning 18 features.
2556
before val loop
after clf fit
after clf val predict
Iteration 2: Features=9 | Best Val Score=0.5943
   >>> Score drop > 0.01 (Val: 0.5943 vs Max: 0.6408). Keeping previous Champion.
Iteration 2: Converged (No features dropped).

Pruning Complete. Reverting to Champion Feature Set (Size: 27) with Score: 0.6408

Grid Search & Selection Complete. Best Parameters: {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 20}

--- Final Test Set Evaluation for CEVA ---
              precision    recall  f1-score   support

           0       0.64      0.72      0.68      1503
           1       0.66      0.58      0.62      1443

    accuracy                           0.65      2946
   

C:\Users\allth\AppData\Local\Temp\ipykernel_23628\770689073.py:28: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ts_numeric = tp_timestamps.astype('datetime64[s]').astype('int64')


Using threshold: 0.03 for CRDO with lead_ptile 15
before while true
2556
before val loop
after clf fit
after clf val predict
Iteration 1: Features=27 | Best Val Score=0.7426
   >>> New Champion Selected! (Score: 0.7426 is within 0.01 of Max 0.7426)
Iteration 1: Pruning 19 features.
2556
before val loop
after clf fit
after clf val predict
Iteration 2: Features=8 | Best Val Score=0.5555
   >>> Score drop > 0.01 (Val: 0.5555 vs Max: 0.7426). Keeping previous Champion.
Iteration 2: Converged (No features dropped).

Pruning Complete. Reverting to Champion Feature Set (Size: 27) with Score: 0.7426

Grid Search & Selection Complete. Best Parameters: {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 20}

--- Final Test Set Evaluation for CRDO ---
              precision    recall  f1-score   support

           0       0.47      0.61      0.53      1238
           1       0.64      0.50      0.56      1708

    accuracy                           0.55      2946
   

...

C:\Users\allth\AppData\Local\Temp\ipykernel_23628\770689073.py:28: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ts_numeric = tp_timestamps.astype('datetime64[s]').astype('int64')


Using threshold: 0.03 for DIOD with lead_ptile 85
before while true
2551
before val loop
after clf fit
after clf val predict
Iteration 1: Features=27 | Best Val Score=0.6152
   >>> New Champion Selected! (Score: 0.6152 is within 0.01 of Max 0.6152)
Iteration 1: Pruning 15 features.
2551
before val loop
after clf fit
after clf val predict
Iteration 2: Features=12 | Best Val Score=0.5648
   >>> Score drop > 0.01 (Val: 0.5648 vs Max: 0.6152). Keeping previous Champion.
Iteration 2: Converged (No features dropped).

Pruning Complete. Reverting to Champion Feature Set (Size: 27) with Score: 0.6152

Grid Search & Selection Complete. Best Parameters: {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 20}

--- Final Test Set Evaluation for DIOD ---
              precision    recall  f1-score   support

           0       0.79      0.78      0.79      1717
           1       0.70      0.72      0.71      1224

    accuracy                           0.75      2941
  

C:\Users\allth\AppData\Local\Temp\ipykernel_23628\770689073.py:28: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ts_numeric = tp_timestamps.astype('datetime64[s]').astype('int64')


✅ Model saved to: ./models/champion_model_DIOD_[0.79334917 0.69689737].pkl
GCTS

================ Starting analysis for symbol: GCTS ================
Converting timestamp_level_SPY_GCTS to datetime...
Using threshold: 0.03 for GCTS with lead_ptile 15
before while true
2551
before val loop
after clf fit
after clf val predict
Iteration 1: Features=27 | Best Val Score=0.5563
   >>> New Champion Selected! (Score: 0.5563 is within 0.01 of Max 0.5563)
Iteration 1: Pruning 15 features.
2551
before val loop
after clf fit
after clf val predict
Iteration 2: Features=12 | Best Val Score=0.8857
   >>> New Champion Selected! (Score: 0.8857 is within 0.01 of Max 0.8857)
Iteration 2: Converged (No features dropped).

Pruning Complete. Reverting to Champion Feature Set (Size: 12) with Score: 0.8857

Grid Search & Selection Complete. Best Parameters: {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 20}

--- Final Test Set Evaluation for GCTS ---
              precision   

C:\Users\allth\AppData\Local\Temp\ipykernel_23628\770689073.py:28: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ts_numeric = tp_timestamps.astype('datetime64[s]').astype('int64')


In [17]:
results_df.to_csv('results_giant_1_3.csv')

In [ ]:
#MODEL TRAINING 3: SAVE MODEL
import joblib
import os

# --- NEW: SAVE CHAMPION MODEL ---
# Create directory if it doesn't exist
MODEL_DIR = "./models/"
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

# Bundle the model and the specific features used
model_data = {
    'model': final_clf,
    'features': current_features,  # These are the champion features
    'params': best_params,
    'symbol': s,
    'side': p  # '15' for downside, '85' for upside
}

# Save the file
save_path = f"{MODEL_DIR}champion_model_{s}_{p}.pkl"
joblib.dump(model_data, save_path)

print(f"✅ Model saved to: {save_path}")

{'VSH', 'GCTS', 'LSCC', 'TSEM', 'MCHP', 'NXPI', 'INTC', 'DIOD', 'MX', 'SPY'}
Features for DIOD ([0.88221709 0.5414673 ]):


In [ ]:
#REAL-TIME 1: MONITOR PRICES
#REAL_TIME 2: CONFIGURE STATIC/DYNAMIC SPLIT AND PREDICT FXN
# Add this to the top of your real-time cell (before importing asyncio or alpaca)
#!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()
import asyncio
import datetime as dt
import pandas as pd
import numpy as np
import joblib
import re
import time
from alpaca.data.live.stock import StockDataStream
import os
# ==========================================
# 1. THE REAL-TIME ENGINE CLASS
# ==========================================
class TradingModelRealTime:
    def __init__(self, model_pkl_path, static_features_csv_path):
        self.data = joblib.load(model_pkl_path)
        self.model = self.data['model']
        self.feature_order = self.data['features']
        self.symbol = self.data['symbol']
        self.dynamic_map = self._parse_feature_requirements()
        
        # Load the "Baseline" (The last row of your weekly CSV)
        full_df = pd.read_csv(static_features_csv_path)
        self.baseline_row = full_df.iloc[-1].to_dict()

    def _parse_feature_requirements(self):
        dynamic_map = []
        cl_pattern = r"change_lag_(\d+)(\d+)th_ptile_(.*)"
        zs_pattern = r"z_score_(\d+)_(.*)"
        for feat in self.feature_order:
            m_cl = re.match(cl_pattern, feat)
            m_zs = re.match(zs_pattern, feat)
            if m_cl:
                lag, ptile, ticker = m_cl.groups()
                dynamic_map.append({'feature': feat, 'type': 'cl', 'ticker': ticker,
                                  'ancillary': [f'{ptile}th_ptile_lag_{lag}days_{ticker}']})
            elif m_zs:
                lag, ticker = m_zs.groups()
                dynamic_map.append({'feature': feat, 'type': 'zs', 'ticker': ticker,
                                  'ancillary': [f'rolling_mean_{lag}_{ticker}', f'rolling_std_{lag}_{ticker}']})
        return dynamic_map

    def predict(self, live_prices):
        spy_price = live_prices.get('SPY')
        if not spy_price: return None
        
        input_vector = {}
        # Fill static features
        for feat in self.feature_order:
            input_vector[feat] = self.baseline_row.get(feat, 0)

        # Calculate dynamic features
        for item in self.dynamic_map:
            ticker = item['ticker']
            if ticker in live_prices:
                rel_price = live_prices[ticker] / spy_price
                if item['type'] == 'cl':
                    thresh = self.baseline_row[item['ancillary'][0]]
                    input_vector[item['feature']] = (rel_price - thresh) / thresh
                elif item['type'] == 'zs':
                    mu = self.baseline_row[item['ancillary'][0]]
                    sigma = self.baseline_row[item['ancillary'][1]]
                    input_vector[item['feature']] = (rel_price - mu) / sigma if sigma != 0 else 0

        X = pd.DataFrame([input_vector])[self.feature_order]
        return self.model.predict_proba(X)[0][1]

# ==========================================
# 2. GLOBAL CONFIG & INITIALIZATION
# ==========================================
API_KEY = "AKH3ZFJ2LWAKH7SSV2AHKD4PK4"
SECRET_KEY = "7gGJDh5UAxh3gg8V21d3AcuPfNJcwtKqukiqMhdhsjrW"
MODEL_PATH ='models/champion_model_DIOD_[0.88221709 0.5414673 ].pkl'
CSV_PATH = 'feature_presentation_semiconductors.csv'
PREDICTION_INTERVAL = 10  # Run prediction every 10 seconds

# Global dictionary to store incoming prices
live_prices = {}

# Initialize Engine
engine = TradingModelRealTime(MODEL_PATH, CSV_PATH)

# Auto-generate Watchlist from Model Features
base_name = os.path.basename(MODEL_PATH) # Gets 'champion_model_DIOD_85.pkl'
target_stock = base_name.split('_')[2]
watchlist = set(f.split('_')[-1] for f in engine.feature_order)
watchlist.update([target_stock, 'SPY'])
WATCHLIST = list(watchlist)

# ==========================================
# 3. ASYNC TASKS
# ==========================================
async def on_quote_update(data):
    """Updates the price dictionary as fast as quotes arrive."""
    if data.bid_price > 0 and data.ask_price > 0:
        live_prices[data.symbol] = (data.bid_price + data.ask_price) / 2

async def inference_loop():
    """Calculates predictions on a fixed timer (N seconds)."""
    print(f"[INFO] Inference loop started (Interval: {PREDICTION_INTERVAL}s)")
    while True:
        # 1. Ensure we have SPY and at least some data before predicting
        if 'SPY' in live_prices and len(live_prices) > 1:
            try:
                prob = engine.predict(live_prices)
                print(prob)
                timestamp = dt.datetime.now().strftime('%H:%M:%S')
                print(f"[{timestamp}] Model: {engine.symbol} | Signal Probability: {prob:.4f}")
                
                # Logic for trading:
                if prob > 0.85:
                    print(f"*** HIGH CONVICTION SIGNAL DETECTED FOR {engine.symbol} ***")
                    
            except Exception as e:
                print(f"[ERROR] Prediction failed: {e}")
        else:
            print(f"[WAIT] Waiting for more data... (Current count: {len(live_prices)}/{len(WATCHLIST)})")
            
        await asyncio.sleep(PREDICTION_INTERVAL)

async def main():
    while True:
        inference_task = None
        try:
            # 1. Start Fresh
            stream = StockDataStream(API_KEY, SECRET_KEY)

            # 2. Subscribe
            for symbol in WATCHLIST:
                stream.subscribe_quotes(on_quote_update, symbol)
            
            print(f"[START] Attempting connection at {dt.datetime.now()}")

            # 3. Start the Inference Loop as a background task
            inference_task = asyncio.create_task(inference_loop())
            
            # 4. Run the stream (This is the "main" blocking call)
            # We await this directly. If it fails, it jumps to 'except'
            await stream.run()

        except Exception as e:
            print(f"[RECONNECT] Error caught in main loop: {e}")
            
            # 5. Mandatory Cleanup
            if inference_task:
                inference_task.cancel()
                try:
                    await inference_task
                except asyncio.CancelledError:
                    pass
            
            print("Retrying in 5 seconds...")
            await asyncio.sleep(5)

# --- JUPYTER EXECUTION ---
# If you are in a notebook, just run this:
await main()

[START] Attempting connection at 2026-01-14 08:48:51.794951
[INFO] Inference loop started (Interval: 10s)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data... (Current count: 0/10)
[WAIT] Waiting for more data..

In [11]:
import scipy
print("SciPy version:", scipy.__version__)

import scipy.sparse
print("Has scipy.sparse.linalg?", hasattr(scipy.sparse, 'linalg'))

import sklearn
print("sklearn version:", sklearn.__version__)

SciPy version: 1.13.1
Has scipy.sparse.linalg? False


AttributeError: module 'scipy.sparse' has no attribute 'linalg'

In [11]:
import pandas as pd
CSV_PATH = 'feature_presentation_semiconductors.csv'
df=pd.read_csv(CSV_PATH)
[c for c in df.columns if 'rolling_mean_VSH' in c]

[]

In [2]:


# --- EXAMPLE USAGE ---
engine = TradingModelRealTime('models/champion_model_GCTS_[0.82748538 0.77298961].pkl', 'feature_presentation_semiconductors.csv')
live_data = {'DIOD': 85.50, 'VSH': 24.20, 'SPY': 502.10}
probability = engine.predict(live_data)
print(f"Current Signal Probability: {probability:.4f}")

Current Signal Probability: 0.9747


In [ ]:
# BACKTEST MODE: SIMULATE REAL-TIME STREAMING WITH HISTORICAL DATA
import asyncio
import datetime as dt
import pandas as pd
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockQuotesRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

# Reuse your existing on_quote_update, inference_loop, etc.
# Assume engine, WATCHLIST, etc., are defined from prior cell

API_KEY = "AKH3ZFJ2LWAKH7SSV2AHKD4PK4"
SECRET_KEY = "7gGJDh5UAxh3gg8V21d3AcuPfNJcwtKqukiqMhdhsjrW"

client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# Config: Simulate "real-time" from historical data
SIM_START_DATE = dt.datetime(2025, 12, 1)  # Start of backtest period
SIM_END_DATE = dt.datetime(2025, 12, 2)   # End of backtest period
SIM_SPEED = 60.0  # Acceleration factor (1.0 = real-time pace; 60.0 = 1 sim minute per real second)

async def simulate_stream():
    # Fetch historical quotes for the period (use quotes for bid/ask mid simulation)
    request_params = StockQuotesRequest(
        symbol_or_symbols=WATCHLIST,
        start=SIM_START_DATE,
        end=SIM_END_DATE,
        timeframe=TimeFrame(amount=15, unit=TimeFrameUnit.Minute)  # Or adjust to your level
    )
    print('after alpaca data call')
    historical_quotes = client.get_stock_quotes(request_params).df.reset_index()
    print('after get_stock_quotes')
    # Group by timestamp to simulate "incoming" data points
    grouped = historical_quotes.groupby('timestamp')
    
    print(f"[SIM] Backtesting from {SIM_START_DATE} to {SIM_END_DATE} with speed {SIM_SPEED}x")
    
    # Start inference loop in background
    inference_task = asyncio.create_task(inference_loop())
    
    try:
        for timestamp, group in grouped:
            sim_time = timestamp.to_pydatetime()
            print(f"[SIM] Processing data at {sim_time}")
            print('input stock prices? ',grouped)
            # Simulate quote updates for each symbol in this timestamp
            for _, row in group.iterrows():
                synthetic_quote = type('Quote', (), {})()  # Mock quote object
                synthetic_quote.symbol = row['symbol']
                synthetic_quote.bid_price = row['bid_price']
                synthetic_quote.ask_price = row['ask_price']
                await on_quote_update(synthetic_quote)
            
            # Delay to simulate real-time (adjusted by speed)
            await asyncio.sleep(60 / SIM_SPEED)  # 60s/minute divided by speed
        
        print("[SIM] Backtest complete")
    
    finally:
        inference_task.cancel()
        try:
            await inference_task
        except asyncio.CancelledError:
            pass

# Run the sim (use nest_asyncio if in Jupyter)
import nest_asyncio
nest_asyncio.apply()
await simulate_stream()

after alpaca data call
after get_stock_quotes
[SIM] Backtesting from 2025-12-01 00:00:00 to 2025-12-02 00:00:00 with speed 60.0x
[SIM] Processing data at 2025-12-01 09:00:00.000738+00:00
input stock prices?  <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001D9B4CABD10>
[INFO] Inference loop started (Interval: 10s)
[WAIT] Waiting for more data... (Current count: 0/10)
[SIM] Processing data at 2025-12-01 09:00:00.000759+00:00
input stock prices?  <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001D9B4CABD10>
[SIM] Processing data at 2025-12-01 09:00:00.002018+00:00
input stock prices?  <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001D9B4CABD10>
[SIM] Processing data at 2025-12-01 09:00:00.002033+00:00
input stock prices?  <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001D9B4CABD10>
[SIM] Processing data at 2025-12-01 09:00:00.002098+00:00
input stock prices?  <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001D9B4CABD